# Attributes
Each object in SHOP comprises mulitple attributes that defines the properties of the object, and where the results can be aquired from.

## Types
Each object type has it individual attributes. Once SHOP is running, we can query information about available attribute types and more detailed infomration about the indivudal attribute. Since the documentation is integrated into the SHOP source code, we can query the attribute documentaion online and the documentation will always be the most recent available. Execute the code below to list all attribute types for an object type: 

In [ ]:
from check.attributes import *
import pandas as pd
from pyshop import ShopSession
shop = ShopSession()
starttime = pd.Timestamp('2022-05-09')
endtime = pd.Timestamp('2022-05-10')
shop.set_time_resolution(starttime=starttime, endtime=endtime, timeunit='hour')

In [ ]:
shop.model.reservoir.get_attribute_names()

Once an object has been created, we can query more detailed information about the respective attributes.

In [ ]:
rsv1 = shop.model.reservoir.add_object("Reservoir1")

For example, the datatype_dict shows what datatype the respective attributes represent.

In [ ]:
rsv1.datatype_dict

 You can also get more information about each individual attribute by calling the `info()` function:

In [ ]:
rsv1.water_value_input.info()

### Exercise
Complete the code below:
- Create a new plant named `Plant1`
- Check the info for the following attributes: `outlet_line`, `discharge`, `intake_loss` and `penstock_loss`.

In [ ]:
shop.model.plant

## Data types
All attributes represent one of the following datatypes:
- string
- string_array
- int
- int_array
- double
- double_array
- txy
- xy
- xyt
- xy_array
- sy

### Primitive data types
The first six data types are so-called primited datatypes. They are either a single instance of the primitive datatypes string, integer and double or an array of these (string is actually a list of characters but will be treated as a primited datatype in pySHOP). These are either represented using standard python `int`, `float` and `str`, and python `list` of these, or we can use the equivalent [numpy](https://numpy.org/) objects.

#### String
This datatype represents a simple string, for example the name of the log-file. This type is only used rare occasions.
Example:
```python
a = 'My string'
```

#### String array
String arrays represent an array of strings, and the order of the strings is of importance. This datatype is mainly used in the `lp_model` functionality the describe the variables and constraints.
```python
a = ['My string1', 'My string2']
```
or
```python
import numpy as np
b = np.array(['My string1', 'My string2'])
```

#### Int
Integer values, typically flag values and ids.
```python
a = 5
```

#### Int array
Array of integers.
```python
a = [1, 3, 7]
```
or
```python
import numpy as np
b = np.array([1, 3, 7])
```

#### Double
Decimal number typically describing phyiscal property of object.
```python
a = 5.7
```

#### Double array
Array of numbers describing physical properties of object.
```python
a = [1.1, 3.1, 7.5]
```
or
```python
import numpy as np
b = np.array([1.1, 3.1, 7.5])
```

### Advanced data types
The remaining data types are index data and will be represented using [pandas](https://pandas.pydata.org/) which is an open source library for managing structured data. In pySHOP, we are using the [pandas.Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html) object, which is a one-dimensional array with axis labels that also can represent time. The examples below shows how to create simple `pandas.Series`. The pandas objects also have a convenient built-in plotting function to visualize the data with the index along the x-axis and the value along the y-axis:

In [ ]:
import pandas as pd
xy = pd.Series(
    index=[0, 2, 3],
    data=[3, 7, 5]
)
xy.plot()

In [ ]:
txy = pd.Series(
    index=[pd.Timestamp('2022-05-09'), pd.Timestamp('2022-05-10')],
    data=[5.3, 9.2]
)
txy.plot()

#### Txy
`txy` represents time series data where the index is a sequence of timestamps and the data are numbers. If the time series has less datapoints than the timeresolution of SHOP, forward fill interpolation and extrapolation is used. In other words, the missing values are replaced by the first previously known value. This is the most widely used data type in SHOP and is used for both time series of floats and integers, such as flags. It is used for both input and output data.
```python
a = pd.Series(
    index=[starttime, starttime+pd.Timedelta(hours=6)],
    data=[0, 5]
)
```

#### Xy
`xy` represent simple tables where the index is a number, for example generator efficiency and the volume to head releation for reservoirs. This data type can be interpreted as a simple 2D curve.
```python
a = pd.Series(
    index=[0, 2, 3],
    data=[2.2, 1.1, 0.3]
)
```

#### Xyt
`xyt` is time indexed `xy`. At each time step t, there exist an corresponding table `xy`. This data type is mainly used for marginal cost curves and bid curves that are updated for each time step. See best profit example on SHOP portal for further details.

#### Xy_array
`xy_array`, also known as `xyn`, is similar to `xyt`, but is indexed by a reference value `n` and not time. It is, for example, used to describe turbine efficiency where different reference head levels have their respective efficiency curve where `x` denote the discharge and `y` the efficiency. This data type can be interpreted as a 3D surface. Note that the reference value for each `xy` curve is given by the `name`-field in each `pd.Series` object.
```python
a = [
    pd.Series(
        name=50
        index=[0, 10],
        data=[90, 95]
    ),
    pd.Series(
        name=60
        index=[0, 10],
        data=[92, 97]
    )
]
```

#### Sy
`sy` is similar with `xy`, but the index `s` is a string instead of a number. The type is mainly used to describe the busbar PTDF factors for power flow calculations in SHOP.

### Exercise
Create a loop that prints the attribute names and datatypes for reservoirs.
(The solution is shown when running the function `show_solution`.)

In [ ]:
# Write you own loop here

In [ ]:
show_solution()

## Getting and setting attributes
Attribute values can be read using the attribute `get()` function, and set with the `set(...)` function as shown in the example below:

In [ ]:
inflow = pd.Series(
    index=[starttime, starttime + pd.Timedelta(hours=6)],
    data = [50, 100]
)
rsv1.inflow.set(inflow)
rsv1.inflow.get()

Since `txy` attributes are so common in SHOP, there exists some additional pySHOP functionality for setting constant `txy` attributes. A constant `txy` attribute can be set by supplying a single (float) value to the `set` function. The code below overwrites the inflow values we just added to the reservoir object, and instead assigns a constant inflow value of 25 m3/s for all time steps: 

In [ ]:
rsv1.inflow.set(25.0)
rsv1.inflow.get()

### Exercise
- Create a plant `Plant1` and a generator `Gen1`.
- Set the plant `penstock_loss` to `[0.001,0.002]`.
- Set the plant `production_schedule` to 0 from the starttime and then 10 from hour 6.
- Set the generator `gen_eff_curve` to 90% at 10 MW, 93% at 20MW and 95% at 30MW.

In [ ]:
plant1 = shop.model.plant.add_object("Plant1")
***

check1(shop)

## Video solution

<div style="padding:56.25% 0 0 0;position:relative;"><iframe src="https://player.vimeo.com/video/778452374?h=002f166a9b&badge=0&autopause=0&player_id=0&app_id=58479/embed" allow="autoplay; fullscreen; picture-in-picture" allowfullscreen frameborder="0" style="position:absolute;top:0;left:0;width:100%;height:100%;"></iframe></div>